In [ ]:
import h5py
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
%matplotlib inline

In [ ]:
node1 = 'sonycnode-b827eb86d458.sonyc'
node2 = 'sonycnode-b827eb329ab8.sonyc'
node3 = 'sonycnode-b827ebb40450.sonyc'
node4 = 'sonycnode-b827eb73e772.sonyc'

path1 = f'../sonyc/spl/2017/{node1}.h5'
path2 = f'../sonyc/spl/2017/{node2}.h5'
path3 = f'../sonyc/spl/2017/{node3}.h5'
path4 = f'../sonyc/spl/2017/{node4}.h5'

In [ ]:
train_start, train_end = pd.Timestamp('2017-04-01'), pd.Timestamp('2017-07-01')
test_start, test_end = pd.Timestamp('2017-07-01'), pd.Timestamp('2017-08-01')

In [ ]:
def load_spl_data(path):
    df = pd.read_hdf(path, key='minute_intervals')
    df = df[df['timestamp'] < 1514764800] # remove any 2018 values
    df['datetime[utc]'] = pd.to_datetime(df['timestamp'], unit='s')
    
    return df[['datetime[utc]', 'dBAS_mean']]

def reduce_spl_data(df, start, end):
    # reduced data to mean of a given hourly interval
    reduced_df = df[
        (start <= df['datetime[utc]']) & 
        (df['datetime[utc]'] < end) &
        (df['datetime[utc]'].dt.minute == 51)
    ].reset_index(drop=True)
    
    grouped_df = reduced_df.groupby([
        reduced_df['datetime[utc]'].dt.month,
        reduced_df['datetime[utc]'].dt.day,
        reduced_df['datetime[utc]'].dt.hour]
    ).mean()
    
    new_date_col = (grouped_df
        .index
        .to_series()
        .apply(lambda t: pd.Timestamp(*((2017,) + t + (51,))))
        .reset_index(drop=True))
    
    grouped_df.reset_index(drop=True, inplace=True)
    grouped_df['datetime[utc]'] = new_date_col

    return grouped_df[['datetime[utc]', 'dBAS_mean']]

# load spl stuff
spl1_df = load_spl_data(path1)
reduced_spl1_df = reduce_spl_data(spl1_df, train_start, test_end)

In [ ]:
# load weather stuff
weather_df = pd.read_csv(
    '../data/weather.csv', 
    usecols=['datetime[utc]', 'precipitation[mm]'], 
    parse_dates=['datetime[utc]']
)

reduced_weather_df = weather_df[
    (train_start <= weather_df['datetime[utc]']) & 
    (weather_df['datetime[utc]'] < test_end) &
    (weather_df['datetime[utc]'].dt.minute == 51) # only using this rolling window
].reset_index(drop=True)

In [ ]:
# TODO: verify later that each hour is also represented
working_df = pd.merge(reduced_weather_df, reduced_spl1_df, how='outer', on='datetime[utc]').sort_values(by='datetime[utc]')

if any(working_df['precipitation[mm]'].isna()):
    print(f"{len(working_df[working_df['precipitation[mm]'].isna()])} NaNs in 'precipitation[mm],' replacing them with 0.0")
    working_df.loc[working_df[working_df['precipitation[mm]'].isna()].index, 'precipitation[mm]'] = 0.0
    
if any(working_df['dBAS_mean'].isna()):
    avg = working_df['dBAS_mean'].dropna().mean()
    print(f"{len(working_df[working_df['dBAS_mean'].isna()])} NaNs in 'dBAS_mean,' replacing them with {avg}")
    working_df.loc[working_df[working_df['dBAS_mean'].isna()].index, 'dBAS_mean'] = avg

In [ ]:
plotting_data = working_df[
    (pd.Timestamp('2017-04-01') <= working_df['datetime[utc]']) & 
    (working_df['datetime[utc]'] < pd.Timestamp('2017-05-01'))
]
plotting_data.plot(x='precipitation[mm]', y='dBAS_mean', kind='scatter')

In [ ]:
working_df.plot(x='precipitation[mm]', y='dBAS_mean', kind='scatter')

In [ ]:
# add rained class
working_df['rained'] = (working_df['precipitation[mm]'] > 0.00).astype(int)

In [ ]:
# assumption: time independent
training_cla_df = working_df[
    (train_start <= working_df['datetime[utc]']) & 
    (working_df['datetime[utc]'] < train_end)
][['rained', 'dBAS_mean']]

testing_cla_df = working_df[
    (test_start <= working_df['datetime[utc]']) & 
    (working_df['datetime[utc]'] < test_end)
][['rained', 'dBAS_mean']]

training_reg_df = working_df[
    (train_start <= working_df['datetime[utc]']) & 
    (working_df['datetime[utc]'] < train_end)
][['precipitation[mm]', 'dBAS_mean']]

testing_reg_df = working_df[
    (test_start <= working_df['datetime[utc]']) & 
    (working_df['datetime[utc]'] < test_end)
][['precipitation[mm]', 'dBAS_mean']]

In [ ]:
X_train, y_train = (
    training_cla_df['dBAS_mean'].to_numpy().reshape(-1, 1), 
    training_cla_df['rained'].to_numpy()
)
X_test, y_test = (
    testing_cla_df['dBAS_mean'].to_numpy().reshape(-1, 1), 
    testing_cla_df['rained'].to_numpy()
)

### Logistic Regression Classification

In [ ]:
clf_logistic = LogisticRegression()
clf_logistic.fit(X_train, y_train)

In [ ]:
clf_logistic.score(X_test, y_test)

### SVM Classification

In [ ]:
svm = SVC(gamma='auto', class_weight='balanced')
param_grid = {
    'C': list(range(10, 31)),
}
clf_svm = GridSearchCV(svm, param_grid, cv=5)
clf_svm.fit(X_train, y_train)

In [ ]:
clf_svm.best_params_

In [ ]:
clf_svm.score(X_test, y_test)

In [ ]:
print(classification_report(y_test, clf_svm.predict(X_test)))

### Random Forest Classification

In [ ]:
rf = RandomForestClassifier(max_depth=2, class_weight='balanced')
param_grid = {
    'max_depth': [2, 4, 8, 16, 32],
    'n_estimators': [100, 200, 300, 400]
}
clf_rf = GridSearchCV(rf, param_grid, cv=5)
clf_rf.fit(X_train, y_train)

In [ ]:
clf_rf.score(X_test, y_test)

In [ ]:
clf_rf.best_params_

In [ ]:
print(classification_report(y_test, clf_rf.predict(X_test)))